In [1]:
import numpy as np
np.random.seed(42)
import pandas as pd
import re
import os
os.environ['OMP_NUM_THREADS'] = '4'
from tqdm import tqdm

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score

from keras import regularizers
from keras.models import Model
from keras.layers import Dropout, Input, Dense, Embedding, SpatialDropout1D, concatenate, PReLU
from keras.layers import CuDNNGRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.optimizers import Nadam, Adam
from keras.preprocessing import text, sequence
from keras.callbacks import Callback, TensorBoard, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.models import load_model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
from keras.engine import InputSpec, Layer
import tensorflow as tf
from global_variables import TRAIN_FILENAME, TEST_FILENAME, SAMPLE_SUBMISSION_FILENAME
from preprocess_utils import preprocess
import warnings
warnings.filterwarnings('ignore')

max_features = 120000
maxlen = 200


#fix4->fix5:去掉concatenate后的Dropout层，SpatialDropout1D百分比0.45->0.5
#fix5->fix6:modelcheckpoint
#fix6->fix7:preprocess, gru后+prelu和dropout etc.
#fix7->fix8:LR decrese, glove twitter200d
#fix8->fix9:10fold
#fix9->fix11:dropout0.5->0.2, attention layer

def glove_preprocess(text):
    #adapted from https://nlp.stanford.edu/projects/glove/preprocess-twitter.rb
    # Different regex parts for smiley faces
    eyes = "[8:=;]"
    nose = "['`\-]?"
    text = re.sub("https?:* ", "<URL>", text)
    text = re.sub("www.* ", "<URL>", text)
    text = re.sub("\[\[User(.*)\|", '<USER>', text)
    text = re.sub("<3", '<HEART>', text)
    text = re.sub("[-+]?[.\d]*[\d]+[:,.\d]*", "<NUMBER>", text)
    text = re.sub(eyes + nose + "[Dd)]", '<SMILE>', text)
    text = re.sub("[(d]" + nose + eyes, '<SMILE>', text)
    text = re.sub(eyes + nose + "p", '<LOLFACE>', text)
    text = re.sub(eyes + nose + "\(", '<SADFACE>', text)
    text = re.sub("\)" + nose + eyes, '<SADFACE>', text)
    text = re.sub(eyes + nose + "[/|l*]", '<NEUTRALFACE>', text)
    text = re.sub("/", " / ", text)
    text = re.sub("[-+]?[.\d]*[\d]+[:,.\d]*", "<NUMBER>", text)
    text = re.sub("([!]){2,}", "! <REPEAT>", text)
    text = re.sub("([?]){2,}", "? <REPEAT>", text)
    text = re.sub("([.]){2,}", ". <REPEAT>", text)
    pattern = re.compile(r"(.)\1{2,}")
    text = pattern.sub(r"\1" + " <ELONG>", text)
    return text

def deduplicate(x, threshold):
    word_list = x.split()
    num_words = len(word_list)
    if num_words == 0:
        return x
    else:
        num_unique_words = len(set(word_list))
        unique_ratio = num_words/num_unique_words
        if unique_ratio > threshold:
            x = ' '.join(x.split()[:num_unique_words])
        return x

train = pd.read_csv(TRAIN_FILENAME)
test = pd.read_csv(TEST_FILENAME)
submission = pd.read_csv(SAMPLE_SUBMISSION_FILENAME)

train['comment_text'] = train['comment_text'].apply(lambda x: glove_preprocess(x))
test['comment_text'] = test['comment_text'].apply(lambda x: glove_preprocess(x))

train = preprocess(train)
train = preprocess(train)

X_train = train["comment_text"].fillna(" ").values
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_test = test["comment_text"].fillna(" ").values

tokenizer_w = text.Tokenizer(num_words=max_features)
tokenizer_w.fit_on_texts(texts=list(X_train) + list(X_test))

X_train = tokenizer_w.texts_to_sequences(X_train)
X_test = tokenizer_w.texts_to_sequences(X_test)

x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)
del X_train, X_test

def get_coefs(word, *arr): 
    return word, np.asarray(arr, dtype='float32')

# EMBEDDING_FILE = '../embed/crawl-300d-2M.vec'
EMBEDDING_FILE = 'assets/embedding_models/glove/glove.twitter.27B.200d.txt'
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in tqdm(open(EMBEDDING_FILE, encoding='UTF-8')))

word_index = tokenizer_w.word_index 
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, idx in tqdm(word_index.items()):
    if idx >= max_features: 
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix[idx] = embedding_vector


/home/christof/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/christof/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


preprocessing
lowercase
removing breaks
expanding contractions
replacing smileys
replacing ip
removing links
replacing numbers
removing bigrams
isolating punct
preprocessing
lowercase
removing breaks
expanding contractions
replacing smileys
replacing ip
removing links
replacing numbers
removing bigrams
isolating punct
fitting tokenizer
getting embeddings
done
